In [85]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchsummary import summary

import numpy as np
import PIL
import os

In [86]:
def changeFileName(path, targetName):
  fileNames = os.listdir(path)
  i = 1
  for name in fileNames:
      src = os.path.join(path, name)
      dst = targetName + '.' + str(i) + '.jpg'
      dst = os.path.join(path, dst)
      os.rename(src, dst)
      i += 1

In [87]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
dataDir = './dataset/AI-DATASET/data'


train_dir = os.path.join(dataDir, 'train')
test_dir = os.path.join(dataDir, 'test')

./dataset/AI-DATASET/data\train
./dataset/AI-DATASET/data\test


In [88]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, files, root, mode='train', transform=None):
        self.files = files
        self.root = root
        self.mode = mode
        self.transform=transform
        
        if 'beverage' in files[0]:
            self.label = 0
        elif 'dailynecessity' in files[0]:
            self.label = 1
        elif 'Foods' in files[0]:
            self.label = 2
        elif 'iceCream' in files[0]:
            self.label = 3
        elif 'noodles' in files[0]:
            self.label = 4
        elif 'Snack' in files[0]:
            self.label = 5
    
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, index):
        img = PIL.Image.open(os.path.join(self.root, self.files[index]))
        
        if self.transform:
            img = self.transform(img)
        if self.mode == 'train':
            return img, np.array([self.label])
        else:
            return img, self.files[index]

In [ ]:
model = torchvision.models.resnet50(pretrained=True)

num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(num_ftrs, 1024),
    nn.Dropout(0.2),
    nn.Linear(1024, 512),
    nn.Dropout(0.1),
    nn.Linear(512, 1),
    nn.Sigmoid()
)

model.cuda()
summary(model, input_size=(3,224,224))


In [ ]:
train_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((256,256)),
    torchvision.transforms.RandomCrop(224),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
])
test_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224,244)),
    torchvision.transforms.ToTensor(),
])
